<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/drop_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm -U

     |████████████████████████████████| 1.2MB 2.4MB/s 
  Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
import gc
import pickle
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

PATH =  '/content/drive/My Drive/M5_Forecasting/data/'
PATH_MODEL = '/content/drive/My Drive/M5_Forecasting/kaggle_kernels/models/'

SEED = 13
random.seed(SEED)
np.random.seed(SEED)
lgb.__version__

'2.3.1'

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
VER = 1

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse'    
    'verbose': -1,
    'n_estimators': 1000,
    }

lgb_params['seed'] = SEED              

In [ ]:
#@title read_data not for share

def read_data(store, start = 0):
  """
  read data by store_id
  """
  mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

  remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d','sales']

  df = pd.concat([
                pd.read_pickle(PATH + 'grid_part_1.pkl'),
                pd.read_pickle(PATH + 'grid_part_2.pkl').iloc[:, 2:],
                pd.read_pickle(PATH + 'grid_part_3.pkl').iloc[:, 2:]],
                axis =1
                )
  df= df[df.store_id == store]
  gc.collect()
  encode = pd.read_pickle(PATH + 'mean_encoding_df.pkl').iloc[:, 2:]
  encode = encode[encode.index.isin(df.index)]
  df = pd.concat([df, encode], axis=1)
  del encode
  gc.collect()
  
  lags = pd.read_pickle(PATH + 'lags_df_28.pkl').iloc[:, 3:]
  lags = lags[lags.index.isin(df.index)]
  df = pd.concat([df, lags], axis=1)
  del lags
  gc.collect()

  #new add stocks
  stocks = pd.read_pickle(PATH + "wmt_stocks_CloseVolumeAdj.pkl")
  stocks = stocks[stocks.index.isin(df.index)]
  df = pd.concat([df, stocks], axis=1)
  del stocks
  gc.collect()

  
  features = [col for col in list(df) if col not in remove_features]
  df = df[['id','d','sales']+features]

  #skip row
  df = df[df.d >= start].reset_index(drop=True)
  return df, features 

In [ ]:
def read_data(store, start = 0):

    #тут убираем  почему разбиваем
    #STORES&DEPT
    remove_features = [
                       'id',                        
                       'date',
                       'wm_yr_wk',
                       'd',
                       'sales',

                       'store_id',
                       'state_id',
                       
                       ]

    
    df = pd.concat([
                pd.read_pickle('grid_part_1.pkl'),
                pd.read_pickle('grid_part_2.pkl').iloc[:, 2:],
                pd.read_pickle('grid_part_3.pkl').iloc[:, 2:]],
                axis =1)
    df= df[df.store_id == store]  
    gc.collect()
    
   
    lags = pd.read_pickle('lags_df_default28.pkl').iloc[:, 3:]
    lags = lags[lags.index.isin(df.index)]
    df = pd.concat([df, lags], axis=1)
    del lags
    gc.collect()
    
    encode = pd.read_pickle(PATH + 'mean_encoding_df.pkl').iloc[:, 2:]
    encode = encode[encode.index.isin(df.index)]
    df = pd.concat([df, encode], axis=1)
    del encode
    gc.collect()
    
    #new add stocks
    stocks = pd.read_pickle("wmt_stocks_CloseVolumeAdj.pkl")
    stocks = stocks[stocks.index.isin(df.index)]
    df = pd.concat([df, stocks], axis=1)
    del stocks
    gc.collect()
    
    #new add ewm, exp
    ewm = pd.read_pickle(PATH + "lags_ewm_exp.pkl").iloc[:, 2:]
    ewm = ewm[ewm.index.isin(df.index)]
    df = pd.concat([df, ewm], axis=1)
    del ewm
    gc.collect()

    df_16 = pd.read_pickle('df_16.pkl')
    df_16 = df_16[df_16.index.isin(df.index)]
    df = pd.concat([df, df_16], axis=1)
    del df_16
    gc.collect()

    df_16_stat = pd.read_pickle('df_16_stat.pkl')
    df_16_stat = df_16_stat[df_16_stat.index.isin(df.index)]
    df = pd.concat([df, df_16_stat], axis=1)
    del df_16_stat
    gc.collect()

    df_16_prob = pd.read_pickle('df_16_prob.pkl')
    df_16_prob = df_16_prob[df_16_prob.index.isin(df.index)]
    df = pd.concat([df, df_16_prob], axis=1)
    del df_16_prob
    gc.collect()

    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d','sales']+features]
    
    #skip row
    df = df[df.d >= start].reset_index(drop=True)
    return df, features


In [ ]:
gc.collect()

0

subsample data

In [ ]:
#subsample for test
store = 'CA_4'

df, features = read_data(store, start = 0) #~800Mb, #subsamp_df.info()#48Mb

In [ ]:
# #'1,2,3'.split(',', maxsplit=1)
# 'CA_2 TX_3 WI_1 CA_4'.split(' ', maxsplit= 3)

In [ ]:
#@title train_subsample

def train_subsamp(df, features):
  """
  make for test, and make viz features  
  """

  train_mask = df.d <= 1913
  val_mask = (df.d <= 1913) & (df.d > (1913-28))
  
  train_data = lgb.Dataset(df[train_mask][features], 
                        label=df[train_mask]['sales'])
  train_data.save_binary('train_data.bin')
  train_data = lgb.Dataset('train_data.bin')
  val_data = lgb.Dataset(df[val_mask][features], 
                      label=df[val_mask]['sales'])  
  del df
  gc.collect()
  print('Start train lgbm')
  estimator = lgb.train(lgb_params,
                        train_data,
                        valid_sets = [val_data],
                        verbose_eval = 100,
                        )
  !rm '/content/train_data.bin'
  del train_data, val_data#, estimator

  gc.collect()
  print('Ends') 
  return estimator

In [ ]:
# #CA_1
# bad = ['enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
#        'enc_dept_id_std', 'enc_state_id_cat_id_mean',
#        'enc_state_id_cat_id_std', 'enc_state_id_dept_id_mean',
#        'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
#        'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
#        'enc_store_id_dept_id_std']

# bad_2 = ['price_momentum', 'event_name_2', 'event_type_2']
# features_01  = [f for f in features if f not in bad+bad_2]

# #TX_2
# bad_3 = ['price_momentum', 'event_name_2', 'event_type_2']
# bad_4 = ['cat_id', 'event_type_1', 'enc_dept_id_mean', 'enc_dept_id_std',
#        'enc_state_id_cat_id_mean', 'enc_state_id_cat_id_std',
#        'enc_state_id_dept_id_mean', 'enc_state_id_dept_id_std',
#        'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std']
# features  = [f for f in features if f not in bad+bad_2]
# #wi_3
# bad_5 = ['price_momentum', 'event_name_2', 'event_type_2']
# bad_6 = ['cat_id', 'enc_dept_id_mean', 'enc_dept_id_std',
#        'enc_state_id_dept_id_mean', 'enc_state_id_dept_id_std',
#        'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std']

# #not sub
# bad_7 = ['cat_id', 'price_nunique', 'price_momentum', 'price_momentum_y',
#        'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
#        'snap_CA', 'snap_TX', 'snap_WI', 'quarter', 'weekend',
#        'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
#        'enc_dept_id_std', 'enc_state_id_cat_id_mean',
#        'enc_state_id_cat_id_std', 'enc_state_id_dept_id_mean',
#        'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
#        'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
#        'enc_store_id_dept_id_std', 'num_lags28', 'num_lags29', 'num_lags30',
#        'num_lags31', 'num_lags32', 'num_lags33', 'num_lags34', 'num_lags35',
#        'num_lags36', 'num_lags37', 'num_lags38', 'num_lags39', 'num_lags40',
#        'num_lags41', 'num_lags42', 'rolling_num_7_mean', 'rolling_num_14_mean',
#        'num_lags_30_roll_7', 'num_lags_30_roll_14']

# bads = bad +bad_2+bad_3+bad_4+bad_5+bad_6 + bad_7
#to drops maybe not drop cat_id
# 'cat_id'

# 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
# 'enc_dept_id_std', 'enc_state_id_cat_id_mean', 'enc_state_id_cat_id_std',
# 'enc_state_id_dept_id_mean', 'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
# 'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std'

# 'event_name_2', 'event_type_1', 'event_type_2'

# 'price_momentum'
################################################DROP##########################################
# #CA_4
# bad = ['cat_id', 'weekend', 'enc_dept_id_mean_item_id',
#        'enc_dept_id_std_item_id', 'enc_state_id_dept_id_mean_item_id',
#        'enc_state_id_dept_id_std_item_id', 'enc_store_id_dept_id_mean_item_id',
#        'enc_store_id_dept_id_std_item_id', 'enc_state_id_mean',
#        'enc_state_id_std', 'enc_store_id_mean', 'enc_store_id_std',
#        'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
#        'enc_dept_id_std']

# #TX_1
# bad_2 = ['cat_id', 'price_momentum', 'event_type_1', 'event_name_2',
#        'event_type_2', 'weekend', 'enc_dept_id_mean_item_id',
#        'enc_dept_id_std_item_id', 'enc_state_id_cat_id_mean_item_id',
#        'enc_state_id_cat_id_std_item_id', 'enc_state_id_dept_id_mean_item_id',
#        'enc_state_id_dept_id_std_item_id', 'enc_store_id_cat_id_mean_item_id',
#        'enc_store_id_cat_id_std_item_id', 'enc_store_id_dept_id_mean_item_id',
#        'enc_store_id_dept_id_std_item_id', 'enc_state_id_mean',
#        'enc_state_id_std', 'enc_store_id_mean', 'enc_store_id_std',
#        'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
#        'enc_dept_id_std']
# #WI_1
# bad_3 = ['cat_id', 'price_momentum', 'event_type_1', 'event_name_2',
#        'event_type_2', 'enc_dept_id_mean_item_id', 'enc_dept_id_std_item_id',
#        'enc_store_id_cat_id_mean_item_id', 'enc_store_id_cat_id_std_item_id',
#        'enc_store_id_dept_id_mean_item_id', 'enc_store_id_dept_id_std_item_id',
#        'enc_state_id_mean', 'enc_state_id_std', 'enc_store_id_mean',
#        'enc_store_id_std', 'enc_cat_id_mean', 'enc_cat_id_std',
#        'enc_dept_id_mean', 'enc_dept_id_std']
# #no

# bad_4 = ['cat_id', 'price_momentum', 'event_type_1', 'event_name_2',
#        'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'weekend',
#        'enc_dept_id_mean_item_id', 'enc_dept_id_std_item_id',
#        'enc_state_id_dept_id_mean_item_id', 'enc_state_id_dept_id_std_item_id',
#        'enc_store_id_dept_id_mean_item_id', 'enc_store_id_dept_id_std_item_id',
#        'enc_state_id_mean', 'enc_state_id_std', 'enc_store_id_mean',
#        'enc_store_id_std', 'enc_cat_id_mean', 'enc_cat_id_std',
#        'enc_dept_id_std']

# bads = bad + bad_2+ bad_3+bad_4
#############################################DROP###########################################
# #CA_2
# bad = ['cat_id', 'price_nunique', 'snap_CA', 'snap_TX', 'snap_WI']
# #TX_3
# bad_2 = ['cat_id', 'snap_WI', 'weekend', 'enc_state_id_cat_id_mean_item_id',
#          'enc_state_id_cat_id_std_item_id']
# #WI_1
# bad_3 = ['cat_id', 'price_nunique', 'snap_CA']
# #CA_4
# bad_4 = ['cat_id', 'price_nunique', 'snap_CA', 'snap_TX', 'snap_WI', 'quarter',
#        'weekend', 'num_lags29', 'num_lags30', 'num_lags31', 'num_lags32',
#        'num_lags33', 'num_lags34', 'num_lags35', 'num_lags36', 'num_lags37',
#        'num_lags38', 'num_lags39', 'num_lags41']

# bads = bad + bad_2 + bad_3 + bad_4

In [ ]:
#numpy.unique(ar, return_index=False, return_inverse=False, return_counts=False, axis=None)[source]
bads_un, count_un = np.unique(bads, return_counts=True)
temp = {}
for i, j in zip(bads_un, count_un):
  temp[i] = j

In [ ]:
list_bads = pd.DataFrame([temp]).T
list_bads[list_bads[0] > 1].index

Index(['cat_id', 'price_nunique', 'snap_CA', 'snap_TX', 'snap_WI', 'weekend'], dtype='object')

In [ ]:
drop = ['event_name_2', 'event_type_1', 'event_type_2', 'price_momentum',
        'cat_id', 'price_nunique', 'snap_CA', 'snap_TX', 'snap_WI', 'weekend']
keep_coll = [i for i in features if i not in drop]
features = keep_coll

In [ ]:
perm_features = ['item_id',
                 'dept_id',
                 'ewm_lags7',
                 'num_lags_7_roll_7',
                 'num_lags_7_roll_14', 
                 'num_lags_7_roll_30',
                 'enc_store_id_mean_item_id',
                 'dayofweek'
                 ]
features = perm_features

In [ ]:
features

In [ ]:
subsamp = np.array_split(list(df['id'].unique()), 10)
score_features = pd.DataFrame()
scores = []
for sub in subsamp:
  subsamp_df = df[df['id'].isin(sub)].reset_index(drop=True)
  model = train_subsamp(subsamp_df, features)
  scores.append(model.best_score['valid_0']['rmse'])
  score_features = pd.concat([score_features, pd.Series(model.feature_importance())], axis =1)
  pd.DataFrame(model.feature_importance(), index=features).plot(kind='bar',figsize=(17, 5))

In [ ]:
"""
Subsample:
  -CA_2
    ['event_name_2', 'event_type_1', 'event_type_2', 'price_momentum'] --> (1.7802324353191772, 0.45158946277682527)
  -TX_3 --> (1.605701682659935, 0.4332082446979661)
  -WI_1 --> (1.5142664096872367, 0.4108342572650494)
Not subsample:
  -CA_4 --> (1.317845, 0.015445123479134565)

Subsample:
  -CA_2
    ['cat_id', 'price_nunique', 'snap_CA', 'snap_TX', 'snap_WI', 'weekend'] --> (1.7823305656292259, 0.45302598267327565)
  -TX_3 --> (1.6078363699461182, 0.43318972019543384)
  -WI_1 --> (1.517839978292082, 0.4119846665658397)
Not subsample:
  -CA_4 --> (1.3203650000000002, 0.014567340060167846)

prom_features:
  -CA_4 --> (1.3153707690346885, 0.3549353095401551)
"""
np.mean(scores), np.std(scores)

(1.3153707690346885, 0.3549353095401551)

In [ ]:
new_features = ['jan',
                'feb',
                'mar',
                'apr',
                'may',
                'jun']
                

[1.4473524758440748,
 0.9656838925609528,
 1.5552063894651775,
 1.4446131082296554,
 1.0496933013205834,
 1.3079983472768146,
 1.0788898776306532,
 1.2061298215866436,
 0.899996070932644,
 0.7732966154654519]

###subsample

In [ ]:
score_features.index = features

In [ ]:
score_features.mean(axis = 1)[score_features.mean(axis = 1) < 50].index#score_features[1:].mean(axis =1).mean()]

Index(['enc_state_id_cat_id_mean_item_id', 'enc_state_id_cat_id_std_item_id'], dtype='object')

###permutation

In [ ]:
def train_permutation(df, features):

  train_mask = df.d <= (1913 -28)
  val_mask = df.d > (1913 -28)
  
  train_data = lgb.Dataset(df[train_mask][features], 
                        label=df[train_mask]['sales'])  
  train_data.save_binary('train_data.bin')
  train_data = lgb.Dataset('train_data.bin')
  val_data = lgb.Dataset(df[val_mask][features], 
                      label=df[val_mask]['sales'])
  #-------------------------  
  # X_test = df[test_mask][features]
  # y_test=df[test_mask]['sales']
  #-------------------------
  X_test = df[val_mask][features]
  y_test=df[val_mask]['sales']
     
  del df
  gc.collect()
  print('Start train lgbm')
  estimator = lgb.train(lgb_params,
                        train_data,
                        valid_sets = [val_data],
                        verbose_eval = 100,
                        #early_stopping_rounds=50                        
                        )
  !rm '/content/train_data.bin'
  del train_data, val_data#, estimator
  gc.collect()
  print('Ends')
  print('Permutation\n')
  
  pred = estimator.predict(X_test)
  rmse = np.sqrt(np.mean(np.square(y_test - pred)))
  print(f'Baseline  with lags RMSE: {rmse}')
  for f in features:
    temp_test = X_test.copy()

    if temp_test[f].dtypes.name != 'category':
          temp_test[f] = np.random.permutation(temp_test[f].values)    
  
    pred = estimator.predict(temp_test[features])
    test_rmse = np.sqrt(np.mean(np.square(y_test - pred)))
    print(f, np.round(test_rmse - rmse, 4)) 

train_permutation(df, features)

Start train lgbm


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	valid_0's rmse: 1.25485
[200]	valid_0's rmse: 1.25299
[300]	valid_0's rmse: 1.24464
[400]	valid_0's rmse: 1.26053
[500]	valid_0's rmse: 1.26232
[600]	valid_0's rmse: 1.26683
[700]	valid_0's rmse: 1.27396
[800]	valid_0's rmse: 1.28039
[900]	valid_0's rmse: 1.2883
[1000]	valid_0's rmse: 1.28293
[1100]	valid_0's rmse: 1.28189
[1200]	valid_0's rmse: 1.27538
[1300]	valid_0's rmse: 1.2755


In [ ]:
Baseline  with lags RMSE: 1.2342131449390197
item_id 0.0
dept_id 0.0
cat_id 0.0
start_sell 0.031
sell_price 0.0002
price_max 0.0005
price_min 0.0003
price_std 0.0003
price_mean 0.0006
price_norm -0.0002
price_nunique -0.0002
item_nunique 0.0012
price_momentum 0.0003
price_momentum_m 0.0009
price_momentum_y 0.0
event_name_1 0.0
event_type_1 0.0
event_name_2 0.0
event_type_2 0.0
snap_CA 0.0
snap_TX 0.0
snap_WI 0.0
day -0.0015
month 0.0491
quarter 0.002
week 0.0125
dayofweek 0.0097
year 0.0
weekend 0.0005
sales_lag_28 0.0
sales_lag_29 -0.0
sales_lag_30 0.0005
sales_lag_31 0.0005
sales_lag_32 0.0004
sales_lag_33 0.0005
sales_lag_34 0.0002
sales_lag_35 -0.0003
sales_lag_36 0.0005
sales_lag_37 0.0002
sales_lag_38 0.0001
sales_lag_39 0.0005
sales_lag_40 0.0002
sales_lag_41 -0.0001
sales_lag_42 -0.0001
rolling_mean_7 0.0008
rolling_std_7 0.0008
rolling_mean_14 0.0005
rolling_std_14 0.0009
rolling_mean_30 0.0005
rolling_std_30 -0.0
rolling_mean_60 0.0005
rolling_std_60 0.0006
rolling_mean_180 0.0016
rolling_std_180 0.0006
rolling_mean_tmp_1_7 0.2515
rolling_mean_tmp_1_14 0.1663
rolling_mean_tmp_1_30 0.0446
rolling_mean_tmp_1_60 0.0034
rolling_mean_tmp_7_7 0.0181
rolling_mean_tmp_7_14 0.0032
rolling_mean_tmp_7_30 0.0017
rolling_mean_tmp_7_60 0.0006
rolling_mean_tmp_14_7 0.0031
rolling_mean_tmp_14_14 0.0016
rolling_mean_tmp_14_30 0.0006
rolling_mean_tmp_14_60 0.0007
enc_item_id_mean -0.0002
enc_item_id_std 0.0002
enc_state_id_mean_item_id 0.0002
enc_state_id_std_item_id 0.0001
enc_store_id_mean_item_id 0.0045
enc_store_id_std_item_id 0.0044
enc_cat_id_mean_item_id -0.0
enc_cat_id_std_item_id -0.0004
enc_state_id_cat_id_mean_item_id -0.0001
enc_state_id_cat_id_std_item_id -0.0007
Close 0.0009
Volume 0.0002
Adj Close 0.0007
ewm_lags7 0.041
ewm_lags14 0.0006
ewm_lags30 0.0004
ewm_lags60 0.0003
exp_lags7 0.0068
exp_lags14 0.0026
exp_lags30 0.0026
exp_lags60 0.0094
std_sells_month 0.0001
var_sells_month 0.02
mean_sells_month 0.1516
jan 0.0195
feb 0.0129
mar 0.0411
apr 0.172
may 0.0009
jun 0.003